In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dataloader as dl
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, stride=1)
        self.norm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 48, 5, 1)
        self.norm2 = nn.BatchNorm2d(48)
        self.fc1 = nn.Linear(9*9*48, 420)
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(420, 10)

    def forward(self, x):
        x = F.relu(self.norm1(self.conv1(x)))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.norm2(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        # flattening the image
        x = x.view(-1, 9*9*48)
        x = self.dropout1(F.relu(self.fc1(x)))
        # final output
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
def count_correct(output,y):
    sum = 0
    for i in range(len(output)):
        y_hat = output[i].argmax()
        if( y_hat == y[i]):
            sum += 1
    return sum

In [4]:
def train(model_conv,epoch,strg="train"):
    data_set = dl.Ocr(strg,transforms=transforms.Compose([
    transforms.ToTensor()]))
    test_set = dl.Ocr("test",transforms=transforms.Compose([
    transforms.ToTensor()]))
    learning_rate = 0.0001
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model_conv.parameters(),learning_rate)
    dataloader = DataLoader(data_set, batch_size=120,shuffle=True)
    
    for i in range(epoch):
        model_conv.train()
        for _,(x,y) in enumerate(dataloader):
                optimizer.zero_grad()
                output = model_conv(x.float())
                loss = loss_fn(output,y)
                print("-",end="")
                loss.backward()
                optimizer.step()
        acc = get_accuracy(model_conv,data_set)
        print(f"   epoch: {i+1} accuracy: {acc}")
    return (model_conv,optimizer)

In [5]:
# saving the model:
def save_checkpoint(model,optimizer,num_epoch,epoch_prev,save_path='../resources/checkpoint.pth'):
    checkpoint = {'epoch':num_epoch + epoch_prev,
                  'state_dict': model.state_dict(),
                  'optimizer' : optimizer.state_dict()}

    torch.save(checkpoint,save_path)

In [6]:
def get_accuracy(model,dat_set):
    model.eval()
    batch_size = 100
    data = DataLoader(dat_set, batch_size=batch_size)
    temp = 0
    for _,(x,y) in enumerate(data):
        predictions = model(x.float())
        temp += count_correct(predictions,y)
    acc = (temp/len(dat_set))*100
    return acc

In [7]:
model = Net()

In [13]:
(model,optimizer) = train(model,1)
# (model,optimizer) = train(model,5,"test")

------------   epoch: 1 accuracy: 99.14833215046131


In [9]:
# (model,optimizer) = train(model,1,"test")

In [14]:
data_set = dl.Ocr("test",transforms=transforms.Compose([
    transforms.ToTensor()]))
get_accuracy(model,data_set)

98.89669007021064

In [15]:
save_checkpoint(model,optimizer,10,0,save_path="../Weights/checkpoint_new2.pth")